In [11]:
import numpy as np

In [12]:
with open("input.txt") as f:
    dataraw = f.read()
dataraw[-5:]

',115\n'

In [2]:
dataraw = """\
1,0,1~1,2,1
0,0,2~2,0,2
0,2,3~2,2,3
0,0,4~0,2,4
2,0,5~2,2,5
0,1,6~2,1,6
1,1,8~1,1,9
"""

In [13]:
data = np.array([ list(map(lambda x:list(map(int, x.split(','))), dat.split('~'))) for dat in dataraw[:-1].split('\n') ])
nblocks = len(data)
data, nblocks

(array([[[  8,   7, 300],
         [  8,   8, 300]],
 
        [[  5,   1,  54],
         [  5,   2,  54]],
 
        [[  7,   9,  28],
         [  8,   9,  28]],
 
        ...,
 
        [[  6,   9, 117],
         [  9,   9, 117]],
 
        [[  9,   0,   6],
         [  9,   3,   6]],
 
        [[  0,   3, 115],
         [  0,   5, 115]]]),
 1493)

In [4]:
def get_world(data):
    world = np.zeros(np.max(data, axis=(0,1))+1)
    print(world.shape)
    for i in range(len(data)):
        block = data[i]
        x1, y1, z1 = block[0]
        x2, y2, z2 = block[1]
        world[x1:x2+1, y1:y2+1, z1:z2+1] = i+1
    return world

In [31]:
def fall_down(world, nblocks):
    new_world = world.copy()
    fell = 0
    for i in np.arange(1, nblocks+1):
        Bx, By, Bz = np.where(world==i)
        #print(i, Bx, By, Bz)
        if 0 in Bz:
            continue
        if np.all(np.isin(new_world[(Bx, By, Bz-1)], [0,i])): # block can fall down or is its own.
            #print(f"block {i} can fall down")
            fell += 1
            new_world[(Bx, By, Bz)] = 0
            new_world[(Bx, By, Bz-1)] = i
    return new_world, fell

In [32]:
def fall_all_down(world, nblocks):
    maxit = 1000
    it = 0
    while True:
        new_world, fell = fall_down(world, nblocks)
        if not fell:
            print("no block fell down anymore")
            break
        else:
            #print("new:", new_world)
            world = new_world.copy()
        it += 1
        if it >= maxit:
            print("reached max it")
            break
        else:
            print(f"{it:4d}", end=["|","\n"][it%25==0])
    return new_world

In [52]:
def is_disintegrateble(fallen_world, i):
    Bx, By, Bz = np.where(fallen_world==i)
    above = fallen_world[(Bx, By, np.max(Bz)+1)]
    above = above[above!=0]
    above = np.unique(above)
    #print(above)
    if len(above)==0:
        return True
    for iblock in above:
        Cx, Cy, Cz = np.where(fallen_world==iblock)
        below = fallen_world[(Cx, Cy, np.min(Cz)-1)]
        below = below[~np.isin(below,[0,i,-1])]
        #print(iblock, below)
        if len(below) >= 1: # if there is an index below that is not the current block index, it is still supported
            return True
    return False

In [50]:
world = get_world(data)
world.shape

(10, 10, 375)


(10, 10, 375)

In [35]:
fallen_world = fall_all_down(world, nblocks) # goes until 185

   1|   2|   3|   4|   5|   6|   7|   8|   9|  10|  11|  12|  13|  14|  15|  16|  17|  18|  19|  20|  21|  22|  23|  24|  25
  26|  27|  28|  29|  30|  31|  32|  33|  34|  35|  36|  37|  38|  39|  40|  41|  42|  43|  44|  45|  46|  47|  48|  49|  50
  51|  52|  53|  54|  55|  56|  57|  58|  59|  60|  61|  62|  63|  64|  65|  66|  67|  68|  69|  70|  71|  72|  73|  74|  75
  76|  77|  78|  79|  80|  81|  82|  83|  84|  85|  86|  87|  88|  89|  90|  91|  92|  93|  94|  95|  96|  97|  98|  99| 100
 101| 102| 103| 104| 105| 106| 107| 108| 109| 110| 111| 112| 113| 114| 115| 116| 117| 118| 119| 120| 121| 122| 123| 124| 125
 126| 127| 128| 129| 130| 131| 132| 133| 134| 135| 136| 137| 138| 139| 140| 141| 142| 143| 144| 145| 146| 147| 148| 149| 150
 151| 152| 153| 154| 155| 156| 157| 158| 159| 160| 161| 162| 163| 164| 165| 166| 167| 168| 169| 170| 171| 172| 173| 174| 175
 176| 177| 178| 179| 180| 181| 182| 183| 184| 185|no block fell down anymore


In [53]:
nD = 0
for i in np.arange(1, nblocks+1):
    isD = is_disintegrateble(fallen_world, i)
    if isD:
        nD += 1
nD

568

In [28]:
# 568 is too high. # 414 too low. should be 482
is_disintegrateble(fallen_world, 1209)

False

In [27]:
fallen_world[:,:,:9]

array([[[   0.,    0.,    0.,    0.,    0.,    0.,   73.,    0.,    0.],
        [1209.,    0.,    0.,    0.,    0., 1228.,    0.,    0.,    0.],
        [1475., 1475., 1475., 1475., 1475., 1228.,  775.,    0.,    0.],
        [   0.,    0.,    0.,    0.,    0.,    0.,  775.,    0.,    0.],
        [  23.,    0.,    0.,    0.,    0.,    0.,  775.,  651.,  227.],
        [  23.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,  227.],
        [  23.,    0.,    0.,  813.,    0.,    0.,    0.,    0.,    0.],
        [1163., 1163.,    0.,    0.,    0.,    0.,    0.,    0.,    0.],
        [   0.,    0.,    0.,    0.,    0.,  554., 1307.,    0.,    0.],
        [   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.]],

       [[   0.,    0.,    0.,    0.,    0.,  119.,   73.,    0.,    0.],
        [1209.,  148.,    0.,    0.,    0.,    0., 1035., 1359.,  719.],
        [   0.,  148.,    0.,    0.,    0.,   82.,    0.,    0.,  719.],
        [   0.,    0.,    0.,    0.,    0.,  210.